In [2]:
import geopandas as gpd

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
blocks = gpd.read_file('data/blocks_CUTTED.geojson')

In [4]:
blocks

,id,geometry
0,0,"POLYGON ((316983.653 6644038.696, 316986.485 6..."
1,1,"POLYGON ((315287.843 6646411.375, 315288.850 6..."
2,2,"POLYGON ((315582.670 6646809.216, 315582.581 6..."
3,3,"POLYGON ((315264.235 6646596.731, 315274.102 6..."
4,4,"POLYGON ((313479.271 6646628.746, 313479.599 6..."
...,...,...
6189,8718,"POLYGON ((364951.367 6632403.641, 365082.578 6..."
6190,8719,"POLYGON ((365055.659 6632100.991, 364837.693 6..."
6191,8720,"POLYGON ((364591.035 6633403.929, 364597.995 6..."
6192,8721,"POLYGON ((364504.029 6632935.898, 364504.962 6..."


In [5]:
services = gpd.read_file('/home/gk/jupyter/GLEB_SERVICES.geojson')

In [6]:
services["type"].unique()


array(['Рекреационная зона', 'Детский сад', 'Школа'], dtype=object)

In [7]:
services.head()

,id,city_id,service_name,type,area,capacity,geometry
0,251497,1,Парк Заячий ремиз,Рекреационная зона,575007,305,POINT (325272.459 6641351.099)
1,145849,1,ГБДОУ Детский сад №127,Детский сад,699,209,POINT (349698.732 6644965.587)
2,145861,1,Детский сад №6,Детский сад,728,175,POINT (348128.477 6647626.880)
3,145887,1,Детский сад №58,Детский сад,410,60,POINT (349595.482 6650517.181)
4,145900,1,Детский сад №23,Детский сад,1278,280,POINT (356334.944 6651721.334)


In [8]:
buildings = gpd.read_file('/home/gk/jupyter/DF_BUILDINGS.geojson')
buildings.head()

,id,population_balanced,building_area,living_area,storeys_count,is_living,geometry
0,139,27,145.871,NaN,2.0,True,POINT (302692.898 6676302.122)
1,140,0,139.242,NaN,NaN,False,POINT (302937.244 6676141.770)
2,141,0,390.244,NaN,NaN,False,POINT (303057.986 6676397.828)
3,142,52,579.998,NaN,1.0,False,POINT (303083.988 6676431.689)
4,143,128,695.979,915.4,1.0,True,POINT (303267.920 6676509.163)


In [9]:
greens = gpd.read_file('/home/gk/jupyter/GLEB_GREENS_TMP.geojson').to_crs(32636)
parkings = gpd.read_file('/home/gk/jupyter/GLEB_PARKINGS_TMP.geojson').to_crs(32636)


In [10]:
greens['geometry'] = greens['geometry'].centroid
greens = greens[['area', 'capacity', 'geometry']]
greens.rename(columns={'area': 'current_green_area', 'capacity': 'current_green_capacity'}, inplace=True)

parkings['geometry'] = parkings['geometry'].centroid
parkings = parkings[['capacity', 'geometry']]
parkings.rename(columns={'capacity': 'current_parking_capacity'}, inplace=True)

In [11]:
greens.head(2)

,current_green_area,current_green_capacity,geometry
0,286,692,POINT (350790.126 6644819.889)
1,9450,725,POINT (361616.789 6652035.377)


In [12]:
parkings.head(2)

,current_parking_capacity,geometry
0,27,POINT (351035.892 6653315.227)
1,33,POINT (351910.038 6662276.904)


In [13]:
buildings['living_area'].fillna(0, inplace=True)
buildings['storeys_count'].fillna(0, inplace=True)
buildings

,id,population_balanced,building_area,living_area,storeys_count,is_living,geometry
0,139,27,145.871,0.000000e+00,2.0,True,POINT (302692.898 6676302.122)
1,140,0,139.242,0.000000e+00,0.0,False,POINT (302937.244 6676141.770)
2,141,0,390.244,0.000000e+00,0.0,False,POINT (303057.986 6676397.828)
3,142,52,579.998,0.000000e+00,1.0,False,POINT (303083.988 6676431.689)
4,143,128,695.979,9.154000e+02,1.0,True,POINT (303267.920 6676509.163)
...,...,...,...,...,...,...,...
152795,254350,0,33566.700,0.000000e+00,0.0,False,POINT (354187.305 6661505.707)
152796,1152813,0,0.000,1.000000e+09,0.0,True,POINT (347909.136 6645775.620)
152797,1152822,0,75.000,7.000000e+01,7.0,False,POINT (347444.043 6645400.235)
152798,1152862,0,0.000,0.000000e+00,0.0,False,POINT (354132.314 6651309.941)


In [14]:
from tqdm import tqdm
tqdm.pandas()

def get_living_area(row):
    if row['living_area']:
        return row['living_area']
    else:
        if row['is_living']:
            if row['storeys_count']:
                if row['building_area']:
                    living_area = row['building_area'] * row['storeys_count'] * 0.7
                    
                    return living_area
                else:
                    return 0
            else:
                return 0
        else:
            return 0
            

def get_living_area_pyatno(row):
    if row['living_area']:
        return row['building_area']
    else:
        return 0



buildings['living_area'] = buildings.progress_apply(lambda x: get_living_area(x), axis=1)
buildings['living_area_pyatno'] = buildings.progress_apply(lambda x: get_living_area_pyatno(x), axis=1)
buildings['total_area'] = buildings['building_area'] * buildings['storeys_count']

100%|██████████| 152800/152800 [00:18<00:00, 8149.85it/s] 


In [15]:
buildings.head()

,id,population_balanced,building_area,living_area,storeys_count,is_living,geometry,living_area_pyatno,total_area
0,139,27,145.871,204.2194,2.0,True,POINT (302692.898 6676302.122),145.871,291.742
1,140,0,139.242,0.0000,0.0,False,POINT (302937.244 6676141.770),0.000,0.000
2,141,0,390.244,0.0000,0.0,False,POINT (303057.986 6676397.828),0.000,0.000
3,142,52,579.998,0.0000,1.0,False,POINT (303083.988 6676431.689),0.000,579.998
4,143,128,695.979,915.4000,1.0,True,POINT (303267.920 6676509.163),695.979,695.979


In [16]:
res2 = gpd.sjoin(blocks, greens, predicate='intersects', how='left').groupby('id').agg(
    {'current_green_capacity': 'sum', 
    'current_green_area': 'sum',
    }
)
res2 = res2.reset_index(drop=True).reset_index(drop=False).rename(columns={'index': 'block_id'})
res2

,block_id,current_green_capacity,current_green_area
0,0,1091.0,219008.0
1,1,452.0,13070.0
2,2,0.0,0.0
3,3,0.0,0.0
4,4,0.0,0.0
...,...,...,...
6189,6189,0.0,0.0
6190,6190,0.0,0.0
6191,6191,0.0,0.0
6192,6192,0.0,0.0


In [17]:
res2.sort_values(by='current_green_capacity')

,block_id,current_green_capacity,current_green_area
3096,3096,0.0,0.0
3746,3746,0.0,0.0
3747,3747,0.0,0.0
3749,3749,0.0,0.0
3751,3751,0.0,0.0
...,...,...,...
4215,4215,127393.0,59449.0
3990,3990,128914.0,88779.0
2157,2157,133795.0,244459.0
2907,2907,179370.0,1464259.0


In [18]:
res3 = gpd.sjoin(blocks, parkings, predicate='intersects', how='left').groupby('id').agg(
    {'current_parking_capacity': 'sum'
    }
)
res3 = res3.reset_index(drop=True).reset_index(drop=False).rename(columns={'index': 'block_id'})
res3

,block_id,current_parking_capacity
0,0,237.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,145.0
...,...,...
6189,6189,0.0
6190,6190,0.0
6191,6191,0.0
6192,6192,0.0


In [19]:
res = gpd.sjoin(blocks, buildings, predicate='intersects', how='left').drop(columns=['id_right', 'index_right']).groupby('id_left').agg(
    {'population_balanced': 'sum', 
    'building_area': 'sum',
    'storeys_count': 'median',
    'total_area': 'sum',
    'living_area': 'sum',
    'living_area_pyatno': 'sum',
    }
)
res = res.reset_index(drop=True).reset_index(drop=False).rename(columns={'index': 'block_id'})
res

,block_id,population_balanced,building_area,storeys_count,total_area,living_area,living_area_pyatno
0,0,0.0,1208.07200,0.0,554.1650,0.00000,0.0000
1,1,671.0,30731.41430,1.0,28729.0055,14550.09477,14106.1824
2,2,48.0,1410.13990,1.0,1685.4792,1100.61476,1090.8619
3,3,10.0,1201.53510,1.5,2258.7426,240.91060,172.0790
4,4,47.0,74589.27342,1.0,80447.2072,1104.90527,1112.1592
...,...,...,...,...,...,...,...
6189,6189,76.0,8893.98770,1.0,9731.4245,3323.61575,4434.4635
6190,6190,53.0,3451.47430,1.0,3222.8518,2148.85636,2920.3098
6191,6191,62.0,3369.93570,1.5,4803.1627,2723.32249,2092.9847
6192,6192,34.0,2943.33470,1.0,2377.6347,1431.72449,2045.3207


In [20]:
import pandas as pd
blocks.reset_index(drop=False, inplace=True)

In [21]:
# blocks=blocks.iloc[:, 2:]

In [22]:
blocks

,index,id,geometry
0,0,0,"POLYGON ((316983.653 6644038.696, 316986.485 6..."
1,1,1,"POLYGON ((315287.843 6646411.375, 315288.850 6..."
2,2,2,"POLYGON ((315582.670 6646809.216, 315582.581 6..."
3,3,3,"POLYGON ((315264.235 6646596.731, 315274.102 6..."
4,4,4,"POLYGON ((313479.271 6646628.746, 313479.599 6..."
...,...,...,...
6189,6189,8718,"POLYGON ((364951.367 6632403.641, 365082.578 6..."
6190,6190,8719,"POLYGON ((365055.659 6632100.991, 364837.693 6..."
6191,6191,8720,"POLYGON ((364591.035 6633403.929, 364597.995 6..."
6192,6192,8721,"POLYGON ((364504.029 6632935.898, 364504.962 6..."


In [23]:
import pandas as pd
result = pd.merge(res, res2)
result = pd.merge(result, res3)

result = gpd.GeoDataFrame(pd.merge(blocks, result, left_on='index', right_on='block_id').drop(columns=['index', 'id']), geometry='geometry')
result.rename(columns={'building_area': 'building_area_pyatno', 'total_area': 'building_area'}, inplace=True)

result['current_industrial_area'] = result['building_area_pyatno'] - result['living_area_pyatno']
result.rename(columns={'population_balanced': 'current_population', 'storeys_count': 'floors', 'living_area_pyatno': 'current_living_area'}, inplace=True)
result['area'] = result['geometry'].area
result.drop(columns=['building_area_pyatno', 'building_area', 'living_area'], inplace=True)
result.head()

,geometry,block_id,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
0,"POLYGON ((316983.653 6644038.696, 316986.485 6...",0,0.0,0.0,0.0000,1091.0,219008.0,237.0,1208.07200,2.738972e+06
1,"POLYGON ((315287.843 6646411.375, 315288.850 6...",1,671.0,1.0,14106.1824,452.0,13070.0,0.0,16625.23190,2.795374e+06
2,"POLYGON ((315582.670 6646809.216, 315582.581 6...",2,48.0,1.0,1090.8619,0.0,0.0,0.0,319.27800,1.771556e+04
3,"POLYGON ((315264.235 6646596.731, 315274.102 6...",3,10.0,1.5,172.0790,0.0,0.0,0.0,1029.45610,2.680521e+05
4,"POLYGON ((313479.271 6646628.746, 313479.599 6...",4,47.0,1.0,1112.1592,0.0,0.0,145.0,73477.11422,2.380600e+06


In [25]:
result.sample(20)

,geometry,block_id,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
1042,"POLYGON ((330703.395 6665995.689, 330726.532 6...",1042,49.0,1.0,2093.2088,0.0,0.0,0.0,568.21000,16118.371326
1169,"POLYGON ((332645.025 6655780.342, 332644.943 6...",1169,35.0,1.0,2406.6033,0.0,0.0,547.0,1411.17600,38648.978625
4563,"POLYGON ((353468.711 6643615.126, 353468.728 6...",4563,0.0,NaN,0.0000,0.0,0.0,0.0,0.00000,5298.627343
4584,"POLYGON ((353401.661 6646300.916, 353401.585 6...",4584,1080.0,5.0,12591.0340,0.0,0.0,0.0,4366.65480,33808.882509
4425,"POLYGON ((352483.441 6644715.537, 352486.517 6...",4425,0.0,0.5,0.0000,0.0,0.0,167.0,6847.14970,36373.739526
4761,"POLYGON ((353910.532 6638009.717, 353910.976 6...",4761,10345.0,1.0,38829.5600,88181.0,77363.0,242.0,12984.30872,405483.759019
4404,"POLYGON ((352391.629 6637833.753, 352410.615 6...",4404,0.0,0.0,0.0000,0.0,0.0,0.0,2547.92610,39533.623683
5582,"POLYGON ((357347.503 6650514.097, 357347.888 6...",5582,0.0,0.0,0.0000,0.0,0.0,0.0,908.49880,46137.728718
4884,"POLYGON ((354403.573 6622296.228, 354403.520 6...",4884,0.0,3.0,0.0000,0.0,0.0,486.0,923.49400,21950.590810
4198,"POLYGON ((350923.302 6663166.062, 350924.484 6...",4198,332.0,2.0,6457.5840,0.0,0.0,876.0,3023.95800,155919.941887


In [101]:
result[result['block_id']==5031]

,geometry,block_id,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
5031,"POLYGON ((355530.803 6622219.700, 355438.651 6...",5031,770.0,1.0,19403.38,0.0,0.0,510.0,5422.0502,70773.009325


In [96]:
result.shape

(6194, 10)

In [24]:
result.to_parquet('GLEB_BLOCKS_AND_BUILDINGS_INFO_2.parquet')

In [100]:
a = gpd.read_parquet('GLEB_BLOCKS_AND_BUILDINGS_INFO_2.parquet')

In [ ]:
result = result.drop(columns=[''])

In [ ]:
res

,geometry,block_id,population_balanced,building_area_pyatno,storeys_count,building_area,living_area,living_area_pyatno
0,"POLYGON ((316983.653 6644038.696, 316986.485 6...",0,0,1208,0,554,0,0
1,"POLYGON ((315287.843 6646411.375, 315288.850 6...",1,671,30731,1,28729,14550,14106
2,"POLYGON ((315582.670 6646809.216, 315582.581 6...",2,48,1410,1,1685,1101,1091
3,"POLYGON ((315264.235 6646596.731, 315274.102 6...",3,10,1202,2,2259,241,172
4,"POLYGON ((313479.271 6646628.746, 313479.599 6...",4,47,74589,1,80447,1105,1112
...,...,...,...,...,...,...,...,...
5522,"POLYGON ((358223.951 6637511.025, 358223.986 6...",5522,76,8894,1,9731,3324,4434
5523,"POLYGON ((358381.003 6638013.349, 358435.312 6...",5523,53,3451,1,3223,2149,2920
5524,"POLYGON ((358322.258 6638349.707, 358206.171 6...",5524,62,3370,2,4803,2723,2093
5525,"POLYGON ((357531.632 6638271.400, 357532.003 6...",5525,34,2943,1,2378,1432,2045


In [33]:
res.to_file('GLEB_BLOCKS_AND_BUILDINGS_INFO.geojson')

In [ ]:
res

In [ ]:
res

,block_id,population_balanced,building_area,storeys_count,living_area
0,0,0,1208.07200,0.0,0.00000
1,1,671,30731.41430,1.0,14550.09477
2,2,48,1410.13990,1.0,1100.61476
3,3,10,1201.53510,1.5,240.91060
4,4,47,74589.27342,1.0,1104.90527
...,...,...,...,...,...
5522,5522,76,8893.98770,1.0,3323.61575
5523,5523,53,3451.47430,1.0,2148.85636
5524,5524,62,3369.93570,1.5,2723.32249
5525,5525,34,2943.33470,1.0,1431.72449


In [34]:
services.head()

,id,city_id,service_name,type,area,capacity,geometry
0,251497,1,Парк Заячий ремиз,Рекреационная зона,575007,305,POINT (325272.459 6641351.099)
1,145849,1,ГБДОУ Детский сад №127,Детский сад,699,209,POINT (349698.732 6644965.587)
2,145861,1,Детский сад №6,Детский сад,728,175,POINT (348128.477 6647626.880)
3,145887,1,Детский сад №58,Детский сад,410,60,POINT (349595.482 6650517.181)
4,145900,1,Детский сад №23,Детский сад,1278,280,POINT (356334.944 6651721.334)


In [35]:
res2 = gpd.sjoin(services, res[['geometry']], predicate='intersects').rename(columns={'index_right':'block_id'}).reset_index(drop=True)

In [36]:
res2.explore()

In [37]:
res2.to_file('GLEB_SERVICES_LINKED_TO_BLOCKS.geojson')

In [39]:
res2

,id,city_id,service_name,type,area,capacity,geometry,block_id
0,251497,1,Парк Заячий ремиз,Рекреационная зона,575007,305,POINT (325272.459 6641351.099),731
1,145849,1,ГБДОУ Детский сад №127,Детский сад,699,209,POINT (349698.732 6644965.587),3394
2,149362,1,Лицей №281,Школа,1372,753,POINT (349627.685 6644988.876),3394
3,145861,1,Детский сад №6,Детский сад,728,175,POINT (348128.477 6647626.880),3035
4,145858,1,Детский сад №3 «Андрейка»,Детский сад,517,246,POINT (348224.728 6647682.720),3035
...,...,...,...,...,...,...,...,...
2091,145882,1,Детский сад №22,Детский сад,704,254,POINT (350742.154 6648060.271),3898
2092,149300,1,Школа №429,Школа,3983,735,POINT (319028.808 6645795.005),213
2093,149553,1,Школа № 409,Школа,1880,570,POINT (352297.994 6621690.492),4004
2094,150028,1,Школа №53,Школа,1823,621,POINT (348543.096 6653205.180),3103


In [47]:
res2[res2.block_id == 5758].explore()